# Chapter 10: Word Ladder

Learn recursive CTEs by solving word ladder puzzles.

In [ ]:
import duckdb
conn = duckdb.connect()

## Common Table Expressions (CTEs)

CTEs let you define named subqueries:

In [ ]:
# Basic CTE
conn.execute("""
    WITH numbers AS (
        SELECT 1 as n
        UNION ALL SELECT 2
        UNION ALL SELECT 3
    )
    SELECT n, n * n as squared FROM numbers
""").df()

## Recursive CTEs

Recursive CTEs have a base case and a recursive case:

In [ ]:
# Generate numbers 1-10
conn.execute("""
    WITH RECURSIVE counter AS (
        -- Base case
        SELECT 1 as n
        
        UNION ALL
        
        -- Recursive case
        SELECT n + 1
        FROM counter
        WHERE n < 10
    )
    SELECT * FROM counter
""").df()

## Word Ladder Logic

First, let's check if two words differ by one letter:

In [ ]:
# Load words
conn.execute("""
    CREATE OR REPLACE TABLE words AS
    SELECT column0 as word FROM read_csv('../inputs/four_letter_words.txt', header=false)
""")

# Check letter difference
conn.execute("""
    SELECT 
        'cold' as w1,
        'cord' as w2,
        (SELECT SUM(CASE WHEN 'cold'[i] != 'cord'[i] THEN 1 ELSE 0 END)
         FROM generate_series(1, 4) as t(i)) as diff
""").df()

In [ ]:
# Find neighbors of 'cold'
conn.execute("""
    SELECT word
    FROM words
    WHERE length(word) = 4
      AND (SELECT SUM(CASE WHEN word[i] != 'cold'[i] THEN 1 ELSE 0 END)
           FROM generate_series(1, 4) as t(i)) = 1
""").df()

In [ ]:
conn.close()